<a href="https://colab.research.google.com/github/20021365-DoQuangHuy/Crawl_score/blob/main/CrawlScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import and constant data
from html.parser import HTMLParser
import requests

cookies = {
    '__RC': '10',
    '__R': '1',
    '__tb': '0',
    '__io_first_source': 'l.facebook.com',
    'G_ENABLED_IDPS': 'google',
    'AviviD_sw_version': '1.0.868.210701',
    '__UF': '-1',
    'AviviD_refresh_uuid_status': '2',
    'fg_lastUpdate': '1625799999749',
    '_io_ht_r': '1',
    'AviviD_show_sub': '1',
    'AviviD_already_exist': '1',
    'ePi.log.contentview': '^[^{^\\^d^\\^:27121116,^\\^v^\\^:^\\^,1093054_1,^\\^^}^]',
    '__io_r': 'google.com',
    '__io_d': '1_1252570832',
    '__io_pr_utm_campaign': '^%^7B^%^22referrerHostname^%^22^%^3A^%^22www.google.com^%^22^%^7D',
    '__io_session_id': '0cc0faf77.8b8b22d7c_1627266978670',
    '__io_nav_state': '^%^7B^%^22current^%^22^%^3A^%^22^%^2Fcontent^%^2FMTA5MzA1NA^%^3D^%^3D.html^%^22^%^2C^%^22currentDomain^%^22^%^3A^%^22thanhnien.vn^%^22^%^2C^%^22previousDomain^%^22^%^3A^%^22^%^22^%^7D',
    '__io_unique_26526': '26',
    '__io_uh': '1',
    '__io_visit_26526': '1',
    '__io_unique_39233': '26',
    '__io_visit_39233': '1',
    'AviviD_waterfall_status': '0',
    'AviviD_token_retake': '0',
    'siteCookie': '685',
    'admpshow': 'd^%^2Ca^%^2Ca1',
    '_gat': '1',
    'page_view': '5',
    
}

headers = {
    'authority': 'thanhnien.vn',
    'sec-ch-ua': '^\\^Chromium^\\^;v=^\\^92^\\^, ^\\^',
    'accept': '*/*',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.55',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://thanhnien.vn/giao-duc/tuyen-sinh/2021/tra-cuu-diem-thi-thpt-quoc-gia.html',
    'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
}

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://thanhnien.vn/ajax/diemthi.aspx?kythi=THPT&nam=2021&city=DDT&text=19009935&top=no', headers=headers, cookies=cookies)


In [ ]:
# Class and functions used
class AlphaCandidateHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.cur_list = []
        self.count = 0
        self.data = -1.0
        self.read = False
        self.sbd = []
        self.sbd_append = False
    def handle_starttag(self, tag, attrs):
      if tag == 'td':
        if self.count == 3:
          self.sbd_append = True
        else:
          self.sbd_append = False

        if self.count >= 6 and self.count <= 16:
          self.read = True
          self.data = -1.0
        else:
          self.read = False
        self.count += 1
    def handle_data(self, data):
      if self.read:
        self.data = float(data)
        self.cur_list.append(self.data)
      elif self.sbd_append:
        self.sbd.append(data)
    def handle_endtag(self, tag):
      if tag == 'td':
        if self.read and self.data == -1.0:
          self.cur_list.append(0.0)
      if tag == 'tr':
        self.count = 0

class alphaCandidate:
    def __init__(self):
      self.storage = []
    def parse_0(self, response):
      parser = AlphaCandidateHTMLParser()
      parser.feed(response.content.decode("utf-8"))
      #print(parser.sbd)
      #print(parser.cur_list)
      for i in range(int(len(parser.cur_list) / 11)):
        score_list = {
          "SBD": "",
          "Toan":0.0,
          "Van": 0.0,
          "Ly":0.0,
          "Hoa": 0.0,
          "Sinh":0.0,
          "Khtn":0.0,
          "Su":0.0,
          "Dia":0.0,
          "GDCD":0.0,
          "Khxh":0.0,
          "Anh":0.0,
        }
        score_list["SBD"] = parser.sbd[i]
        score_list["Toan"] = parser.cur_list[11 * i + 0]
        score_list["Van"] = parser.cur_list[11 * i +1]
        score_list["Ly"] = parser.cur_list[11 * i +2]
        score_list["Hoa"] = parser.cur_list[11 * i +3]
        score_list["Sinh"] = parser.cur_list[11 * i +4]
        score_list["Khtn"] = parser.cur_list[11 * i +5]
        score_list["Su"] = parser.cur_list[11 * i +6]
        score_list["Dia"] = parser.cur_list[11 * i +7]
        score_list["GDCD"] = parser.cur_list[11 * i +8]
        score_list["Khxh"] = parser.cur_list[11 * i +9]
        score_list["Anh"] = parser.cur_list[11 * i +10]
        self.storage.append(score_list)

In [ ]:
# Main function
# SBD khong ton tai:
#   response.content == b'\n'
# Thu tu cac mon:
#   Toán (D1); 
#   Ngữ văn (D2); 
#   Vật lí (D3); 
#   Hóa học (D4); 
#   Sinh học (D5); 
#   KHTN (D6); 
#   Lịch sử (D7); 
#   Địa lí (D8); 
#   GDCD (D9); 
#   KHXH (D10); 
#   Ngoại ngữ (D11)
def getScoreList(ma_tinh, start, end):
  condition = True
  sbd = start
  candidate_list = []
  while condition:
    sbd_str = str(ma_tinh * 10000 + int(sbd / 100))
    params = (
        ('kythi', 'THPT'),
        ('nam', '2021'),
        ('city', 'DDT'),
        ('text', sbd_str),
        ('top', 'no'),
    )
    response = requests.get('https://thanhnien.vn/ajax/diemthi.aspx', headers=headers, params=params, cookies=cookies)
    c = alphaCandidate()
    c.parse_0(response)
    c.storage.reverse()
    candidate_list.extend(c.storage)
    sbd += 100
    if sbd > end:
      condition = False

  candidate_list.reverse()
  return candidate_list


In [ ]:
# Thay giá trị tham số của hàm getScoreList và tên csv file nếu muốn tải file về
# Nếu chưa rõ SBD kết thúc, hãy tìm trên mạng để có. SBD_ket_thuc quá lớn có thể làm nặng thêm dữ liệu trả về
import csv
from google.colab import files
csv_file = "Bac_Ninh_full.csv"
myList = []
try:
  # getScoreList(Ma_tinh, SBD_bat_dau, SBD_ket_thuc)
  myList = getScoreList(19, 1, 200)
  myList.reverse()
  keys = myList[0].keys()
  with open(csv_file, 'w', newline='')  as output_file:
      dict_writer = csv.DictWriter(output_file, keys)
      dict_writer.writeheader()
      dict_writer.writerows(myList)
except IOError:
    print("I/O error")

# Lưu file kết quả về máy
files.download(csv_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(myList)